# First Project - Pharmaceutical Drug Analysis 2011 - 2013

In [ ]:
# Dependencies and Setup 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

# Import API key

# Output File (CSV)

## Perform API Calls

## Convert .txt files to .csv

## Merge Data Sources

## Clean Data 

## Plotting the Data

## Written Analysis 